https://jsdokken.com/fenics22-tutorial/heat_eq.html

In [99]:
from dolfinx import fem, mesh, io, plot, default_scalar_type, la
import dolfinx.fem.petsc as petsc
import numpy as np
from mpi4py import MPI
import ufl

In [100]:
# Define temporal parameters
t = 0  # Start time
T = 1.0  # Final time
num_steps = 50
dt = T / num_steps  # time step size

In [101]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)
V = fem.functionspace(domain, ("Lagrange", 1))

# Create initial condition
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))

un = fem.Function(V)
un.name = "u_n"
un.interpolate(initial_condition)

In [102]:
# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

In [103]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(domain, default_scalar_type(-6))
a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (un + dt * f) * v * ufl.dx

In [104]:
def uD_fun(t):
    return lambda x: 0.5*np.exp(-(2-x[0]-4*t)**2) * np.exp(-(2-x[1]-4*t)**2)

uD = fem.Function(V)
uD.interpolate(uD_fun(0))

boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bcs = [fem.dirichletbc(uD, boundary_dofs)]

In [105]:
compiled_a = fem.form(a)
A = petsc.assemble_matrix(compiled_a, bcs = bcs)
A.assemble()

compiled_L = fem.form(L)
b = fem.Function(V)

uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(initial_condition)

xdmf = io.XDMFFile(domain.comm, "diffusion.xdmf", "w")
xdmf.write_mesh(domain)
xdmf.write_function(uh, t)

In [106]:
from petsc4py import PETSc

solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(PETSc.PC.Type.HYPRE)
pc.setHYPREType("boomeramg")

In [107]:
t = 0
for i in range(num_steps):
    t += dt
    uD.interpolate(uD_fun(t))

    # Assemble RHS
    b.x.array[:] = 0
    petsc.assemble_vector(b.vector, compiled_L)

    # Apply boundary condition
    petsc.apply_lifting(b.vector, [compiled_a], [bcs])
    b.x.scatter_reverse(la.InsertMode.add)
    fem.petsc.set_bc(b.vector, bcs)

    # Solve linear problem
    solver.solve(b.vector, uh.vector)
    uh.x.scatter_forward()

    # Update un
    un.x.array[:] = uh.x.array
    
    xdmf.write_function(uh, t)
xdmf.close()